In [25]:
import requests
from urllib.parse import quote
import base64
import time
from datetime import datetime

In [42]:
class TippiLink:
    def __init__(self, username, password, host="192.168.0.1"):
        self._username = username
        self._password = password
        self._host = host

    def _get_headers(self):
        headers = {
            'Referer': 'http://%s/userRpm/MenuRpm.htm' % self._host,
            'Authorization': self._auth_header(),
        }
        return headers

    def _auth_header(self):
        auth_bytes = bytes(self._username+':'+self._password, 'utf-8')
        auth_b64_bytes = base64.b64encode(auth_bytes)
        auth_b64_str = str(auth_b64_bytes, 'utf-8')

        auth_str = quote('Basic {}'.format(auth_b64_str))
        return auth_str
    
    def _get_mac_from_page(self, pageName):
        
        try:

            response = requests.get('http://%s/userRpm/%s.htm' % (self._host, pageName), headers=self._get_headers())
            content = response.content

            ini = 'hostList = new Array('
            fin = ');'

            texto = str(content)[str(content).index(ini)+len(ini)+2:]
            arrTx = texto[:texto.index(fin)].split("\"")
            arr = [l for l in arrTx if len(l.split('-')) == 6]

            return arr
        
        except:
            return "Error"
    
    def get_all_macs(self):
        page = '1'
        macs = []

        while True:
            arr = self._get_mac_from_page("WlanStationRpm.htm?Page="+page)
            page = str(int(page) + 1)

            if set(arr).issubset(set(macs)):
                break
            else:
                macs.extend(arr)

        mac_address = list(set(macs))

        mac_address = [mac.replace('-', ':').lower() for mac in mac_address]

        return mac_address
    
    def estadisticas(self):
        now = datetime.now()
        timestamp = datetime.timestamp(now)
        ini = 'wlanHostPara = new Array('
        ini1 = 'statistList = new Array('
        fin = ');'

        try:
            # numero de conectados
            response = requests.get('http://%s/userRpm/%s.htm' % (tl._host, 'WlanStationRpm.htm'), headers=tl._get_headers())
            content = response.content
            lista = str(content)[str(content).index(ini)+len(ini):str(content).index(fin)].replace('\\n', '').split(',')
            nconx = lista[5]

            # enviados y recibidos
            response = requests.get('http://%s/userRpm/%s.htm' % (tl._host, 'StatusRpm.htm'), headers=tl._get_headers())
            content = response.content
            texto = str(content)[str(content).index(ini1)+len(ini1)+2:]
            statistList = texto[:texto.index(fin)].split(',')

            b_rec = statistList[0]
            b_env = statistList[1]
            
            return [timestamp, nconx, b_rec, b_env]
        except:
            return "Error"
        
    def restart_router(self):
        params = (('Reboot', 'Reboot'),)
        try:
            res = requests.get('http://%s/userRpm/SysRebootRpm.htm' % self._host,
                                headers=self._get_headers(), params=params)
        except:
            pass
    

In [40]:
tl = TippiLink("luis", "AmE3024OT1", "192.168.66.32")


In [41]:
print(tl.estadisticas())

[' 23', '90849820', ' 182616454']


In [32]:
tl.get_all_macs()

['d0:7a:b5:dc:29:63',
 '80:71:7a:eb:57:43',
 'd0:5b:a8:05:81:f4',
 'dc:bf:e9:b4:0d:19',
 '50:c8:e5:c7:34:8b',
 '3c:05:18:38:6c:b9',
 '58:d9:c3:33:d0:f9',
 'fc:a6:21:36:63:de',
 'c0:3f:d5:d9:b4:49',
 'b8:94:36:5f:52:40',
 '80:fd:7a:81:3f:c4',
 'bc:44:86:90:fc:ca',
 '94:27:90:c8:7a:c9',
 'f8:1a:67:cf:ef:82',
 '94:27:90:5a:8b:e8',
 '74:5c:9f:09:a9:a6',
 '90:f6:52:e1:95:4e',
 '60:1d:91:70:0f:1a',
 '86:35:f1:c3:f3:ea',
 '20:64:32:7b:c8:cd',
 '80:5a:04:dc:32:4d',
 '60:14:66:d0:6c:94',
 'a2:01:3d:d1:f2:33',
 '1c:56:fe:be:59:97',
 '00:0c:e7:70:05:a4',
 'a0:28:ed:6c:ce:0a',
 '38:80:df:b4:fd:dd',
 '00:8a:0f:05:57:66',
 '5c:77:76:fd:ec:6e',
 '60:14:66:d0:6f:fe',
 'fc:a6:21:5e:a1:12']